#Welcome to Part 1 of my Yahoo Finance Web Scraping Project! 

The primary purpose for this Python script is to scrape historic data from Yahoo Finance on a few stocks I personally invest in, then format it into a convenient dataframe with all the relevant information I need. Once I have the information in a Pandas dataframe, I'll upload the info into PowerBi, a data visualization tool, and complete my analysis there.

My preferred trading strategies include selling cash-covered calls and covered puts, which are notably safe choices for investing in options. In order to utilize these strategies, I need to analyze historic financial data.

First off, we need to download the required Python libraries, namely Pandas, NumPy, and yfinance. For those unfamiliar with the yfinance library, allow me to explain. An API, or Application Programming Surface, is an intermediary technology that connects, or allows two different systems to talk to each other. Yahoo Finance used to have an API in the past, but it was discontinued on May 15, 2017. This resulted in the creation of yfinance, an open-source Python library that acts as an API, allowing users to scrape financial data from Yahoo Finance. 

In [1]:
#import libraries

import yfinance as yf
import pandas as pd
import numpy as np


After importing the libraries, I want to create a list of stocks I'm interested in analyzing. Fortunately, yfinance makes this incredibly convenient-I just need to insert the ticker symbols for all the stocks I want between 2 apostrophes, then create a variable, "ticker", that is read by a function called yf.Tickers.

In [2]:
#Stock List

symbols= 'AAPL VOO GOOG TSLA SNAP GPS DIS'
ticker=yf.Tickers(symbols)

Now that yfinance knows what stocks I want info on, let's create the first dataframe (ticker_info) and add some filters, so it looks reasonably legible. 

I set the following filters:

   1) Period of last 3 years
   
   2) Grouping the top index by ticker symbol. Do note the dataframe will be multi-layered (2 column headers)
   
   3) Ignore time zones
   
   4) Remove pre and post market values
   
   5) Include info on stock splits and dividends
   
   6) Round all values to 2 decimal places

In [3]:
#Getting information on tickers & adding filters

ticker_info=ticker.history(period='3y', group_by='ticker', ignore_tz=True, prepost=False, actions=True, rounding=True)

ticker_info.head(5)

[*********************100%***********************]  7 of 7 completed


DIS                                                           \
              Open    High     Low   Close    Volume Dividends Stock Splits   
Date                                                                          
2020-03-09  108.69  110.43  104.28  104.35  26233600       0.0          0.0   
2020-03-10  108.10  111.58  102.43  111.46  24083700       0.0          0.0   
2020-03-11  108.42  108.90  102.86  105.51  24771300       0.0          0.0   
2020-03-12   97.62  100.00   91.64   91.81  40392900       0.0          0.0   
2020-03-13  100.78  102.87   92.25  102.52  40960100       0.0          0.0   

             SNAP                ...       VOO                             \
             Open   High    Low  ... Dividends Stock Splits Capital Gains   
Date                             ...                                        
2020-03-09  11.25  12.13  11.00  ...      0.00          0.0           0.0   
2020-03-10  11.91  11.99  10.99  ...      1.18          0.0           0.0   
2020-03-11  11.57  11.67  10.57  ...      0.00          0.0           0.0   
2020-03-12  10.07  11.13   9.71  ...      0.00          0.0           0.0   
2020-03-13  11.09  11.35  10.78  ...      0.00          0.0           0.0   

             AAPL                                                         
             Open   High    Low  Close     Volume Dividends Stock Splits  
Date                                                                      
2020-03-09  64.65  68.17  64.47  65.25  286744800       0.0          0.0  
2020-03-10  67.93  70.21  66.03  69.94  285290000       0.0          0.0  
2020-03-11  68.00  68.93  66.64  67.52  255598800       0.0          0.0  
2020-03-12  62.74  66.18  60.79  60.85  418474000       0.0          0.0  
2020-03-13  64.93  68.62  62.00  68.14  370732000       0.0          0.0  

[5 rows x 50 columns]

Everything looks good on a surface level so far! I checked the Nasdaq, and there are 252 days/year where the stock market is active. 252 x 3 years = 756. Let's fact check below to make sure there are 756 rows under VOO. 

In [11]:
#Quality check
ticker_info.VOO.Dividends.count()

756

Looks like all the rows are recorded. However, there's still one problem...PowerBi's Power Query can only (comfortably) read in 2D data and right now our dataframe has 2 levels of column headers (3D).

To fix this, we need to:

1) unstack the levels (level=0)
2) Put the Ticker column header into the same level as the Date
3) Reset the index (level=1), so Python realizes we are now dealing with a 2D object

In [13]:
#Flatten multilevel dataframe
ticker_info.stack(level=0).rename_axis(['Date', 'Ticker']).reset_index(level=1).head(5)

,Ticker,Capital Gains,Close,Dividends,High,Low,Open,Stock Splits,Volume
Date,,,,,,,,,
2020-03-09,AAPL,NaN,65.25,0.0,68.17,64.47,64.65,0.0,286744800
2020-03-09,DIS,NaN,104.35,0.0,110.43,104.28,108.69,0.0,26233600
2020-03-09,GOOG,NaN,60.78,0.0,62.74,60.00,60.26,0.0,67308000
2020-03-09,GPS,NaN,10.84,0.0,11.38,10.62,11.32,0.0,8322700
2020-03-09,SNAP,NaN,11.45,0.0,12.13,11.00,11.25,0.0,37469800


For convenience, let's give this newly formatted dataframe a name; 'organized_ticker_info'

In [6]:
#Create a new dataframe
organized_ticker_info=ticker_info.stack(level=0).rename_axis(['Date', 'Ticker']).reset_index(level=1)

organized_ticker_info.head(5)

,Ticker,Capital Gains,Close,Dividends,High,Low,Open,Stock Splits,Volume
Date,,,,,,,,,
2020-03-09,AAPL,NaN,65.25,0.0,68.17,64.47,64.65,0.0,286744800
2020-03-09,DIS,NaN,104.35,0.0,110.43,104.28,108.69,0.0,26233600
2020-03-09,GOOG,NaN,60.78,0.0,62.74,60.00,60.26,0.0,67308000
2020-03-09,GPS,NaN,10.84,0.0,11.38,10.62,11.32,0.0,8322700
2020-03-09,SNAP,NaN,11.45,0.0,12.13,11.00,11.25,0.0,37469800


There's still one more problem we need to address: There's no Date column, only a Date index. If we upload the dataframe as it is, PowerBi won't be able to read in the dates (trust me, I learn this the hard way)!

Fortunately, the solution is quite straightforward-simply create a new Date column, and set it equal to the index.

In [7]:
#Now, let's create a new date column in Power Bi
organized_ticker_info['Date'] = organized_ticker_info.index

organized_ticker_info.head(5)

,Ticker,Capital Gains,Close,Dividends,High,Low,Open,Stock Splits,Volume,Date
Date,,,,,,,,,,
2020-03-09,AAPL,NaN,65.25,0.0,68.17,64.47,64.65,0.0,286744800,2020-03-09
2020-03-09,DIS,NaN,104.35,0.0,110.43,104.28,108.69,0.0,26233600,2020-03-09
2020-03-09,GOOG,NaN,60.78,0.0,62.74,60.00,60.26,0.0,67308000,2020-03-09
2020-03-09,GPS,NaN,10.84,0.0,11.38,10.62,11.32,0.0,8322700,2020-03-09
2020-03-09,SNAP,NaN,11.45,0.0,12.13,11.00,11.25,0.0,37469800,2020-03-09


Finally, looks like we have all of our data all cleaned up! 

Next, we'll feed this Python script into PowerBi, and begin analyzing from there. Thank you for joining me on this web scraping and data cleaning project, and see you all in Part 2!